# Multi-objective Optimization Streamlit Application
- Author: senkin.zhan@datarobot.com

## Summary
This accelerator introduces an approach to build a streamlit application using DataRobot deploments for multi-objective optimization.

This notebook outlines how to:

1. Create multiple projects 
2. Build deployment with best model
3. Build Streamlit application connecting to DataRobot


## Setup

### Install and import libraries

In [10]:
!pip install git+https://github.com/datarobot/dr-apps 

  Cloning https://github.com/datarobot/dr-apps to /tmp/pip-req-build-y3l31ku3
  Running command git clone --filter=blob:none --quiet https://github.com/datarobot/dr-apps /tmp/pip-req-build-y3l31ku3
  Resolved https://github.com/datarobot/dr-apps to commit 7074c6606a5c08dc74f7385257d638986deb9baa
  Installing build dependencies ... -

 \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import datetime as dt
import os
import pickle
import time

from dask import compute, delayed  # For parallelization
import datarobot as dr
from datarobot.models.dataset import Dataset
import numpy as np
import pandas as pd

### Bind variables

In [ ]:
# DataRobot Input Path
input_path = "./multi-objective-optimization-files/"

# credentials
API_URL = "<INSERT Deployment API URL>"
DATAROBOT_API_TOKEN = "<INSERT YOUR DataRobot API Token>"
DATAROBOT_KEY = "<INSERT DataRobot Key>"

credentials = [API_URL, DATAROBOT_API_TOKEN, DATAROBOT_KEY]
with open(input_path + "credentials.pickle", mode="wb") as fo:
    pickle.dump(credentials, fo)

# If use groupkfold input group column ["Group_ID"], if use randomkfold input False
group_col = False

# File name, default is japanese version, if use english input steel_strength_en
file_name = "steel_strength_jp.csv"

# Targets(multiple objective), default is japanese version, if use english input ['Yield strength','Tensile strength','Average strength']
targets = ["降伏強度", "引張強度", "平均強度"]

# Optimization Directions, choose minimize or maximize,max 30 targets
directions = ["maximize", "maximize", "maximize"]

# Streamlit App name
app_name = "multi-objective-optimization-demo"

### Run autopilot on datarobot

In [15]:
%%time


def __run_autopilot(target, df, group_col):
    project = dr.Project.create(df, project_name=target)

    ao = dr.AdvancedOptions()
    if group_col != False:
        pm = dr.GroupCV(holdout_pct=0, partition_key_cols=[group_col], reps=5)
    else:
        pm = dr.RandomCV(holdout_pct=0, reps=5)

    project.analyze_and_model(
        worker_count=-1,
        target=target,
        mode="quick",
        partitioning_method=pm,
        advanced_options=ao,
        max_wait=10000,
    )

    return project


def __wait_for_cv(projects):
    for i in range(len(projects)):
        try:
            project = dr.Project.get(project_id=projects[i - 1].id)
            jobs_list = project.get_all_jobs()
            for job in jobs_list:
                job.wait_for_completion(max_wait=60000)
            print("Project " + targets[i] + " completed running autopilot")
        except:
            print("Project " + targets[i] + " occured error")


df = pd.read_csv(input_path + file_name)
df.to_csv(input_path + "feature.csv", index=False)
df_feature = df.drop(targets, axis=1)

delayed_dr_projects = []
for i in range(len(targets)):
    df_ = df_feature.copy()
    df_[targets[i]] = df[targets[i]]
    temp = delayed(__run_autopilot)(targets[i], df_, group_col)
    delayed_dr_projects.append(temp)

projects = compute(delayed_dr_projects)[0]

__wait_for_cv(projects)

Project 降伏強度 completed running autopilot


Project 引張強度 completed running autopilot


Project 平均強度 completed running autopilot
CPU times: user 1.56 s, sys: 86.3 ms, total: 1.64 s
Wall time: 4min 3s


### Run delopyment on datarobot

In [16]:
%%time


def __run_deployment(project):
    model = project.get_model_records()[0]
    target = project.target
    prediction_server = dr.PredictionServer.list()[0]

    registered_model_version = dr.RegisteredModelVersion.create_for_leaderboard_item(
        model_id=model.id,
        name=target,
    )

    deployment = dr.Deployment.create_from_registered_model_version(
        registered_model_version.id,
        label=target,
        default_prediction_server_id=prediction_server.id,
    )

    return deployment


delayed_dr_projects = []
for i in range(len(projects)):
    temp = delayed(__run_deployment)(projects[i])
    delayed_dr_projects.append(temp)

deployments = compute(delayed_dr_projects)[0]

CPU times: user 665 ms, sys: 50.8 ms, total: 716 ms
Wall time: 4min 52s


### Save deployment ids

In [17]:
deployment_ids = []
for i, d in enumerate(deployments):
    deployment_ids.append(d.id)
    print(d, targets[i], d.id, directions[i])
df_config = pd.DataFrame(
    {
        "Target Name": targets,
        "Deployment ID": deployment_ids,
        "Optimization Direction": directions,
    }
)
df_config.to_csv(input_path + "config.csv", index=False)

Deployment(降伏強度) 降伏強度 666c1fd41eeae168439ff6c3 maximize
Deployment(引張強度) 引張強度 666c2041fdf24d5280e1d2f2 maximize
Deployment(平均強度) 平均強度 666c1fd233e8d26f6b9fea94 maximize


### Build streamlit app on datarobot

In [23]:
%%time

!drapps create -t $DATAROBOT_API_TOKEN -e '[Experimental] Python 3.9 Streamlit' -p $input_path $app_name

Using multi-objective-optimization-demoSource custom application source.
Creating new version for multi-objective-optimization-demoSource custom application source.
Uploading project:  [------------------------------------]    0%

Uploading project:  [####################################]  100%
Starting multi-objective-optimization-demo custom application.
Waiting till app is ready:  [------------------------------------]

Waiting till app is ready:  [-------------------------#----------]

Waiting till app is ready:  [-#----------------------------------]

Waiting till app is ready:  [---------------------------------#--]

...

Waiting till app is ready:  [--#---------------------------------]

Waiting till app is ready:  [-----------------#------------------]

Waiting till app is ready:  [---------------------------------#--]

Waiting till app is ready:  [-----------------#------------------]


Custom application is running: https://app.datarobot.com/custom_applications/666c221de9cac654ca42f51a/
CPU times: user 982 ms, sys: 492 ms, total: 1.47 s
Wall time: 1min 37s
